# INSTRUCTIONS

1. Click "▶ Run" in the toolbar above to start the simulator.
2. Each cell is a patient. The number in the cell is the length of stay.
3. To discharge a patient, click the corresponding cell.
4. To admit a patient, click "New Patient."
5. To advance to the next day, click "Next Day."
6. To start over, click "Reset Table."

# LEGEND
- Blue: Census cap
- Green: New patient
- Red: Most recent new patient

---



In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import re

data = {
    'Riven 1 [20]': [1, 1, 1, 2, 2, 3, 5, 6, 9, 10, 13, 14],
    'Riven 2 [15]': [1, 1, 1, 2, 2, 3, 5, 6, 9, 10, 13, 14],
    'Riven 3 [15]': [1, 1, 1, 2, 2, 3, 5, 6, 9, 10, 13, 14],
    'Riven 4 [15]': [1, 1, 1, 2, 2, 3, 5, 6, 9, 10, 13, 14],
    'Riven 5 [15]': [1, 1, 1, 2, 2, 3, 5, 6, 9, 10, 13, 14],
    'VICP [20]': [1, 1, 1, 2, 2, 3, 5, 6, 9, 10, 13, 14],
}


# Global variable to track the position of the last new patient
last_new_patient_pos = None

# Global dictionary to track the number of discharged patients for each team
discharged_patients_count = {}

# Global dictionary to track the total LOS of discharged patients for each team
discharged_los_total = {}

# Function to extract the census cap from the column name
def extract_census_cap(column_name):
    match = re.search(r'\[(\d+)\]', column_name)
    if match:
        return int(match.group(1))
    else:
        return None

# Function to calculate LOSAC scores
def calculate_losac(dataframe):
    losac_scores = {}

    # Calculate the minimum census cap among all teams
    min_census_cap = min([extract_census_cap(team) for team in dataframe.columns])

    for team in dataframe.columns:
        census_cap = extract_census_cap(team)
        if census_cap is not None:
            team_census = dataframe[team].dropna()
            if team_census.count() < census_cap:
                median_los = team_census.median() if team_census.count() > 0 else 1
                losac_scores[team] = (team_census.count() - (census_cap - min_census_cap)) / median_los
            else:
                losac_scores[team] = float('inf')
    return losac_scores

# Function to handle discharging patients
def discharge_patient(selected_cell):
    global df, discharged_patients_count, discharged_los_total

    # Discharge the selected patient
    team, patient = selected_cell.split(',')
    los = df.at[int(patient), team]
    df.at[int(patient), team] = pd.NA

    # Update discharged patients count and LOS total
    discharged_patients_count[team] = discharged_patients_count.get(team, 0) + 1
    discharged_los_total[team] = discharged_los_total.get(team, 0) + (los if los is not None else 0)

    # Sort and redraw the table
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to admit a new patient
def admit_new_patient(_):
    global df, last_new_patient_pos

    # Calculate LOSAC scores with current census
    losac_scores = calculate_losac(df)

    # Filter out teams at full capacity
    eligible_teams = {team: losac for team, losac in losac_scores.items() if df[team].count() < extract_census_cap(team)}

    if eligible_teams:
        # Find the minimum LOSAC score
        min_losac = min(eligible_teams.values())
        tied_teams = [team for team, losac in eligible_teams.items() if losac == min_losac]

        if len(tied_teams) > 1:
            # Calculate the number of open slots for each tied team
            open_slots = {team: extract_census_cap(team) - df[team].count() for team in tied_teams}
            max_open_slots = max(open_slots.values())
            teams_with_max_open_slots = [team for team, slots in open_slots.items() if slots == max_open_slots]

            if len(teams_with_max_open_slots) > 1:
                # Calculate the mean (not median) of LOS for each tied team
                los_means = {team: df[team].mean() for team in teams_with_max_open_slots}
                max_los_mean = max(los_means.values())
                teams_with_max_los_mean = [team for team, los in los_means.items() if los == max_los_mean]

                if len(teams_with_max_los_mean) > 1:
                    # Randomly select a team if there's still a tie
                    selected_team = random.choice(teams_with_max_los_mean)
                else:
                    selected_team = teams_with_max_los_mean[0]
            else:
                selected_team = teams_with_max_open_slots[0]
        else:
            selected_team = tied_teams[0]

        # Insert new patient for the selected team
        df[selected_team] = df[selected_team].shift(1)
        df.at[0, selected_team] = 1
        last_new_patient_pos = (0, selected_team)
        df[selected_team] = pd.to_numeric(df[selected_team], errors='coerce').sort_values().reset_index(drop=True)
    else:
        print("All teams have reached their census cap or there is an error.")

    # Redraw the table with the updated LOSAC
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to increment the LOS for all patients (start new day)
def increment_day(_):
    global df

    # Increment LOS by 1 for all patients
    df = df.applymap(lambda x: x + 1 if pd.notna(x) else x)

    # Sort each team's list in ascending order of LOS
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))

    # Redraw the table with updated LOSAC
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Make sure to update the create_styled_button function with the new parameters
def create_styled_button(description, pos, is_new_patient, is_green, is_blue):
    button = widgets.Button(description=description, layout=widgets.Layout(width='auto'))
    if is_new_patient:
        button.style.button_color = 'lightcoral'  # Light red color for new patient
    elif is_green:
        button.style.button_color = 'lightgreen'  # Light green for patients with LOS of 1
    elif is_blue:
        button.style.button_color = 'lightblue'   # Light blue for patients below census cap
    return button

# Function to draw the census list table
def draw_table(losac_scores):
    global df, last_new_patient_pos, discharged_patients_count, discharged_los_total

    # Create the table grid with the number of rows in the DataFrame, plus three for the header, LOSAC, and discharged patients tally
    num_rows = df.shape[0]
    table = widgets.GridspecLayout(num_rows + 4, df.shape[1] + 1)

    # Add right-aligned "Team [census cap]", "LOSAC:", and "Discharged:" labels to the left of the first column header
    table[0, 0] = widgets.HTML(value="<div style='text-align: right'>Team [Cap]:</div>")
    table[1, 0] = widgets.HTML(value="<div style='text-align: right'>LOSAC:</div>")
    table[2, 0] = widgets.HTML(value="<div style='text-align: right'>Discharged Count:</div>")
    table[3, 0] = widgets.HTML(value="<div style='text-align: right'>Discharged LOS:</div>")

    # Add a two-line header for each column
    for j, team in enumerate(df.columns):
        table[0, j + 1] = widgets.HTML(value=f"<center>{team}</center>")
        losac_score = losac_scores[team]
        table[1, j + 1] = widgets.HTML(value=f"<center>{losac_score:.2f}</center>")
        discharged_count = discharged_patients_count.get(team, 0)
        table[2, j + 1] = widgets.HTML(value=f"<center>{discharged_count}</center>")
        total_los = discharged_los_total.get(team, 0)
        discharged_los = total_los / discharged_count if discharged_count > 0 else 0
        table[3, j + 1] = widgets.HTML(value=f"<center>{discharged_los:.2f}</center>")

    # Add right-aligned row numbers
    for i in range(num_rows):
        table[i + 4, 0] = widgets.HTML(value=f"<div style='text-align: right'>{i + 1}</div>")

    # Adjust the row index in the loop for the table cells
    for i in range(num_rows):
        for j in range(df.shape[1]):
            cell_value = df.iloc[i, j] if pd.notna(df.iloc[i, j]) else ''
            is_new_patient = ((i, df.columns[j]) == last_new_patient_pos and cell_value == 1)
            is_green = cell_value == 1 and not is_new_patient
            team_census_cap = extract_census_cap(df.columns[j])
            is_blue = i < team_census_cap
            button = create_styled_button(str(cell_value) if pd.notna(cell_value) else ' ',
                                          pos=(i, j),
                                          is_new_patient=is_new_patient,
                                          is_green=is_green,
                                          is_blue=is_blue)
            table[i + 4, j + 1] = button

            button.on_click(lambda btn, x=i, y=j: discharge_patient(f"{df.columns[y]},{x}"))

    display(table)

    # Add "New Patient" and "Next Day" buttons
    new_patient_button = widgets.Button(description="New Patient")
    new_patient_button.on_click(admit_new_patient)
    display(new_patient_button)

    next_day_button = widgets.Button(description="Next Day")
    next_day_button.on_click(increment_day)
    display(next_day_button)

    # Add "Reset Table" button
    reset_table_button = widgets.Button(description="Reset Table")
    reset_table_button.on_click(reset_table)
    display(reset_table_button)


def reset_table(_):
    global df, initial_losac_scores, last_new_patient_pos, discharged_patients_count

    # Reset variables
    last_new_patient_pos = None
    discharged_patients_count = {}
    discharged_los_total = {}

    # Reload the DataFrame from the Excel spreadsheet
    df = pd.DataFrame(data)

    # Initialize DataFrame to have the maximum census cap as the number of rows
    max_census_cap = max([extract_census_cap(team) for team in df.columns])
    df = df.reindex(range(max_census_cap))

    # Sort each team's list in ascending order of LOS
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))

    # Recalculate the LOSAC scores
    initial_losac_scores = calculate_losac(df)

    # Redraw the table
    clear_output(wait=True)
    draw_table(initial_losac_scores)

# Read the initial census
df = pd.DataFrame(data)

# Initialize dataframe to have the maximum census cap as the number of rows
max_census_cap = max([extract_census_cap(team) for team in df.columns])
df = df.reindex(range(max_census_cap))

# Sort each team's list in ascending order of LOS
df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))

# Calculate initial LOSAC scores
initial_losac_scores = calculate_losac(df)

# Draw the initial table
draw_table(initial_losac_scores)

GridspecLayout(children=(HTML(value="<div style='text-align: right'>Team [Cap]:</div>", layout=Layout(grid_are…

Button(description='New Patient', style=ButtonStyle())

Button(description='Next Day', style=ButtonStyle())

Button(description='Reset Table', style=ButtonStyle())